In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data yang digunakan :

In [ ]:
df_event <- read.csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/event.csv', stringsAsFactors = F)
dplyr::glimpse(df_event) # Fungsi glimpse untuk melihat data dari package dplyr(membantu manipualasi data)

## Mengubah kolom created_at menjadi tipe Timestamp

In [ ]:
# Package untuk mengolah tipe data Date /Timestamp
library(lubridate) 

# Mengubah tipe data kolom created_at menjadi years-month-day_hour_minute_second 
# Fungsi ymd_hms dari lubridate
df_event$created_at <- ymd_hms(df_event$created_at)

# Lihat kembali ringkasan datanya
dplyr::glimpse(df_event)

# Data Per-Event
## Summary Event

In [ ]:
library(dplyr)

# Dari data frame kelompokkan data berdasarkan nama_event
# Hitung dengan summarise
df_event %>%
	group_by(nama_event) %>%
    # jumlah_event adalah jumlah teradinya event/baris setiap nama_event dengan fungsi n() untuk melihat jumlah
    # loan(pinjaman) untuk mengetahui jumlah unik loan berdasarkan loan_id
    # investor untuk mengetahui jumlah unik investor_id
	summarise(jumlah_event = n(), loan=n_distinct(loan_id), investor=n_distinct(investor_id))

**investor_register** : Event saat Investor register.
Jumlah event sama dengan unik investor, artinya setiap investor melakukan event ini hanya 1 kali. Jumlah loan hanya 1, ini isinya NA, karena register ini tidak memerlukan loan.

**loan_to_marketplace** : Event saat loan diupload ke marketplace,
Jumlah event sama dengan jumlah loan, artinya setiap loan diupload hanya 1 kali. Jumlah investor hanya 1, ini isi NA, karena saat upload ke marketplace tidak berhubungan dengan investor

**investor_view_loan** : Event saat investor melihat detail loan di marketplace.
Jumlah event nya tidak sama dengan unik loan maupun unik investor, artinya 1 investor dapat melihat loan yang sama beebrapa kali, dan 1 loan bisa dilihat oleh beberapa investor berbeda

**investor_order_loan** : Event saat investor memesan loan, menunggu pembayaran.
Jumlah event nya tidak sama dengan unik loan maupun unik investor, artinya 1 loan bisa dipesan oleh beberapa investor berbeda (jika pemesanan sebelumnya tidak dibayar)

**investor_pay_loan** : Event saat investor membayar loan dari pesanan sebelumnya.
Jumlah Event nya sama dengan unik loan, artinya 1 loan ini hanya bisa dibayar oleh 1 investor. Jumlah investor lebih sedikit daripada jumlah loan artinya 1 investor bisa membeli banyak loan

# Mengubah format data proses investasi loan agar nama event menjadi nama kolom
## Event loan di-upload ke marketplace
Karena tidak berhubungan dengan investor maka di proses sendiri

In [ ]:
library(dplyr)
df_marketplace <- df_event %>% filter(nama_event == 'loan_to_marketplace')%>%
select(loan_id, marketplace = created_at)
df_marketplace

## Event investor melihat detail loan
Untuk event investor melihat detail loan, karena investor bisa melihat detail loan berkali kali maka akan diproses terpisah untuk membuat summary per loan per investor

In [ ]:
library(dplyr)
df_view_loan <- df_event %>% filter(nama_event == 'investor_view_loan') %>% 
group_by(loan_id, investor_id) %>% 
summarise(jumlah_view = n(), pertama_view=min(created_at), terakhir_view=max(created_at))
df_view_loan

## Event investor pesan dan bayar loan
Lalu untuk event **investor_order_loan** dan **investor_pay_loan**, karena unik untuk kombinasi loan_id dan investor_id, maka bisa diproses bersamaan

In [ ]:
library(dplyr)
library(tidyr)
df_order_pay <- df_event%>% filter(nama_event %in% 
                                   c('investor_order_loan', 'investor_pay_loan')) %>% 
spread(nama_event, created_at) %>% 
select(loan_id, investor_id, order=investor_order_loan, pay=investor_pay_loan) 
df_order_pay

## Gabungan Data Loan Investasi

In [ ]:
library(dplyr)
df_loan_invest <- df_marketplace %>% 
# Gunakan fungsi left join karena tidak semua loan dan investor ada di setiap data.frame df_marketplace
left_join(df_view_loan, by='loan_id') %>% 
left_join(df_order_pay, by=c('loan_id','investor_id'))
df_loan_invest

# Analisis Proses Investasi
## Melihat hubungan jumlah view dengan order

In [ ]:
library(dplyr)
library(tidyr)
df_loan_invest %>%
# Membuat status order jika order kosong(is.na(order)) maka status not_order
mutate(status_order = ifelse(is.na(order), 'not_order','order')) %>% 
# Hitung jumlah_view dan status order
count(jumlah_view, status_order) %>% 
# Buat status order menjadi sebuah kolom diisi dengan n(jumlah status order) dan fill=0 jika status order kosong diganti 0
spread(status_order, n, fill = 0) %>%
# Menghitung persentase 
mutate(persen_order = scales::percent(order/(order + not_order)))

Dan ternyata tidak ada pola khusus yang menyatakan hubungan banyaknya view dengan keputusan investor memesan loan tersebut. Hampir merata bahwa lebih dari 85% investor yangs sudah melihat loan akan memesannya.

Untuk Jumlah View 4 atau lebih, karena sangat sedikit event nya maka bisa diabaikan.

## Berapa lama waktu yang dibutuhkan investor untuk pesan sejak pertama melihat detail loan

In [ ]:
library(dplyr)
library(tidyr)
df_loan_invest %>%
  filter(!is.na(order)) %>% 
    # difftime ini merupakan fungsi bawaan (package base) yang digunakan untuk menghitung selisih antara 2 waktu, 
    # disini digunakan units “mins” yang berarti output ditampilkan dalam satuan menit.
  mutate(lama_order_view = as.numeric(difftime(order, pertama_view, units = "mins"))) %>% 
  group_by(jumlah_view) %>% 
  summarise_at(vars(lama_order_view), funs(total = n(), min, median, mean, max)) %>% 
  mutate_if(is.numeric, funs(round(.,2)))

Ternyata mayoritas investor langsung memesan loan ketika membuka detailnya, yakni dibawah 5 menit untuk investor yang melihat detail loan 1 kali saja lalu pesan. Untuk yang membuka 2-4 kali waktunya berkisar 30 menitan. Pada jumlah_view 2 dan 3, karena ada outlier pesan lama sampai jauh dari median, ini membuat nilai rata-ratanya terpengaruh menjadi tinggi, 1 jam lebih.

## Rata- rata waktu pemesanan sejak loan di-upload setiap minggu nya
Pada tahap ini, yang dihitung adalah lama waktu order sejak loan itu pertama di-upload.
Data ini akan dibuat dalam bentu plot mingguan untuk melihat bagaimana tren nya.

In [ ]:
library(dplyr)
library(lubridate)
library(ggplot2)
df_lama_order_per_minggu <- df_loan_invest %>%
filter(!is.na(order)) %>%
# kolom tanggal yang merupakan pembualatan kebawah dari waktu upload ke marketplace dalam satuan minggu
mutate(tanggal = floor_date(marketplace, 'week'),
       #  hitung lama_order sejak di-upload ke marketplace (dalam jam) 
       lama_order = as.numeric(difftime(order, marketplace, units = "hour"))) %>%
group_by(tanggal) %>%
summarise(lama_order = median(lama_order))
ggplot(df_lama_order_per_minggu) +
geom_line(aes(x = tanggal, y = lama_order)) +
theme_bw() +
labs(title = "Rata-rata lama order pada tahun 2020 lebih lama daripada 2019",
x = "Tanggal",
y = "waktu di marketplce sampai di-pesan (jam)")

## Apakah Investor membayar pesanan yang dia buat.
Pada tahap ini, yang ingin dilihat adalah berapa persen pesanan yang dibayar oleh investor.
Data ini akan dibuat dalam bentuk plot mingguan untuk melihat bagaimana tren nya.

In [ ]:
library(dplyr)
library(lubridate)
library(ggplot2)

df_bayar_per_minggu <- df_loan_invest %>% 
  filter(!is.na(order)) %>%
  mutate(tanggal = floor_date(marketplace, 'week')) %>% 
  group_by(tanggal) %>%
  summarise(persen_bayar = mean(!is.na(pay))) 

ggplot(df_bayar_per_minggu) +
  geom_line(aes(x = tanggal, y = persen_bayar)) +
  scale_y_continuous(labels = scales::percent) +
  theme_bw() + 
  labs(title = "Sekitar 95% membayar pesanannya. Di akhir mei ada outlier karena lebaran", 
       x = "Tanggal", 
       y = "Pesanan yang dibayar")

## Waktu yang dibutuhkan investor untuk membayar pesanan
Pada tahap ini, yang dihitung adalah lama waktu pembayaran sejak pesanan dibuat.
Data ini akan dibuat dalam bentuk plot mingguan untuk melihat bagaimana tren nya.

In [ ]:
library(dplyr)
library(lubridate)
library(ggplot2)

df_lama_bayar_per_minggu <- df_loan_invest %>% 
  filter(!is.na(pay)) %>%
    # kolom tanggal yang merupakan pembualatan kebawah dari waktu upload ke order dalam satuan minggu
  mutate(tanggal = floor_date(order, 'week'),
         lama_bayar = as.numeric(difftime(pay, order, units = "hour"))) %>% 
  group_by(tanggal) %>%
  summarise(lama_bayar = median(lama_bayar)) 

ggplot(df_lama_bayar_per_minggu) +
  geom_line(aes(x = tanggal, y = lama_bayar)) +
  theme_bw() + 
  labs(title = "Waktu pembayaran trennya cenderung memburuk, 2x lebih lama dari sebelumnya", 
       x = "Tanggal", 
       y = "waktu di pesanan di bayar(jam) ")

## Kesimpulan
Trend pada tahun 2020 cenderung lebih jelek daripada tahun 2019, hal ini mungkin karena adanya pandemi investor menjadi lebih lama untuk memprtimbangkan invest dimana, dan apakah pesanan yang sudah dibuat mau dibayar atau tidak

# Analisis Summary Investor
## Trend Investor Register

In [ ]:
library(dplyr)
library(lubridate)
library(ggplot2)

df_investor_register <- df_event %>% 
  filter(nama_event=='investor_register') %>%
  mutate(tanggal = floor_date(created_at, 'week')) %>% 
  group_by(tanggal) %>%
  summarise(investor = n_distinct(investor_id)) 

ggplot(df_investor_register) +
  geom_line(aes(x = tanggal, y = investor)) +
  theme_bw() + 
  labs(title = "Investor register sempat naik di awal 2020 namun sudah turun lagi", x = "Tanggal", y = "Investor Register")

## Trend Investor Investasi Pertama
Setelah mendaftar, tujuan selanjutnya untuk investor adalah agar dia bisa invest. Hal ini biasa disebut conversion, yakni ketika user convert menjadi user yang kita harapkan, atau naik ke funnel yang lebih baik.

In [ ]:
library(dplyr)
library(lubridate)
library(ggplot2)

df_investor_pertama_invest <- df_event %>% 
  filter(nama_event == 'investor_pay_loan') %>%
  group_by(investor_id) %>% 
  summarise(pertama_invest = min(created_at)) %>% 
  mutate(tanggal = floor_date(pertama_invest, 'week')) %>% 
  group_by(tanggal) %>% 
  summarise(investor = n_distinct(investor_id)) 

ggplot(df_investor_pertama_invest) +
  geom_line(aes(x = tanggal, y = investor)) +
  theme_bw() + 
  labs(title = "Ada tren kenaikan jumlah investor invest, namun turun drastis mulai Maret 2020", 
       x = "Tanggal", y = "Investor Pertama Invest")

## Cohort Pertama Invest berdasarkan Bulan Register
Pada sebelumnya sudah dihitung bagaimana tren investor baru invest setiap minggunya. Selanjutnya akan dilihat conversion invest berdasarkan bulan register.

In [ ]:
library(dplyr)
library(lubridate)
library(tidyr)

df_register_per_investor <- df_event %>%
  filter(nama_event == 'investor_register') %>% 
  rename(tanggal_register = created_at) %>%  
  mutate(bulan_register = floor_date(tanggal_register, 'month'))  %>%  
  select(investor_id, tanggal_register, bulan_register) 

df_pertama_invest_per_investor <- df_event %>%
  filter(nama_event == 'investor_pay_loan') %>% 
  group_by(investor_id) %>% 
  summarise(pertama_invest = min(created_at)) 

df_register_per_investor %>% 
  left_join(df_pertama_invest_per_investor, by = 'investor_id') %>% 
  mutate(lama_invest = as.numeric(difftime(pertama_invest, tanggal_register, units = "day"))%/%30) %>%  
  group_by(bulan_register, lama_invest) %>% 
  summarise(investor_per_bulan = n_distinct(investor_id)) %>% 
  group_by(bulan_register) %>% 
  mutate(register = sum(investor_per_bulan)) %>% 
  filter(!is.na(lama_invest)) %>% 
  mutate(invest = sum(investor_per_bulan)) %>% 
  mutate(persen_invest = scales::percent(invest/register)) %>% 
  mutate(breakdown_persen_invest = scales::percent(investor_per_bulan/invest)) %>%  
  select(-investor_per_bulan) %>%  
  spread(lama_invest, breakdown_persen_invest) 

Terihat bahwa untuk total register paling banyak adalah di bulan Maret 2020, seperti pada chart sebelumnya, hanya saja dari sebanyak itu sampai saat ini belum ada 2% yang sudah invest, sangat jauh dibandingkan bulan sebelumnya, yang bisa mencapai 7% lebih. yang merupakan conversion rate paling tinggi.

Pada umumnya, hanya 5% investor dari semua investor yang mendaftar akan convert. Dan ketike convert mayoritas mereka melakukannya di bulan pertama (kurang dari 30 hari) sejak registrasi.

## Cohort Retention Invest
Setelah cohort investasi pertama, selanjutnya yang dihitung adalah cohort retention. Yakni apakah investor kembali invest lagi di bulan bulan selanjutnya setelah investasi pertama.

In [ ]:
library(dplyr)
library(lubridate)
library(tidyr)

df_investasi_per_investor <- df_event %>%
  filter(nama_event == 'investor_pay_loan') %>%
  rename(tanggal_invest = created_at) %>% 
  select(investor_id, tanggal_invest)

df_pertama_invest_per_investor %>% 
  mutate(bulan_pertama_invest = floor_date(pertama_invest, 'month'))  %>% 
    # bisa gunakan left_join maupun inner_join karena data investor pada keduanya adalah sama
  inner_join(df_investasi_per_investor, by = 'investor_id') %>%
  mutate(jarak_invest = as.numeric(difftime(tanggal_invest, pertama_invest, units = "day")) %/% 30) %>% 
  group_by(bulan_pertama_invest, jarak_invest) %>%
  summarise(investor_per_bulan = n_distinct(investor_id)) %>%
  group_by(bulan_pertama_invest) %>%
  mutate(investor = max(investor_per_bulan)) %>%
  mutate(breakdown_persen_invest = scales::percent(investor_per_bulan/investor)) %>%
  select(-investor_per_bulan) %>%
  spread(jarak_invest, breakdown_persen_invest) %>% 
  select(-`0`)


Terihat bahwa pada bulan febuari terdapat investor yang melakukan investasi pertama paling banyak dibandingkan bulan lainnya. Akan tetapi kelompok tersebut retention nya jelak dibandingkan yang lain. pada 1 bulan setelah investasi pertama, hanya 16% investor saja yang investasi lagi. Ini hanya setengah dari tren pada bulan bulan sebelumnya, dimana sekitar 30% investor akan invest lagi 1 bulan setelah investasi pertama.

cohort yang paling stabil adalah di bulan Agustus 2019. Di sekitar angka 20% setiap bulannya, alaupun pada bulan ketujuh persentasnya ikut turun juga.

# Kesimpulan
Berdasarkan semua analisis yang telah dilakukan, dapat disimpulkan bahwa :

Secara umum, DQLab Finance sebenarnya sedang dalam growth yang positif, fluktuatif naik turun terjadi karena perbedaan behaviour di tanggal tertentu, yang dipengaruhi oleh hal lain, misalnya gajian.
Pada bulan Maret, April sampai pertangah Mei terjadi banyak penurunan pada metriks yang dianalisis, hal ini mungkin karena adanya pandemi Covid19, perlu dianalisis lebih lanjut apakah memang karena itu.
Secara Umum, 5% dari total investor yang register setiap bulannya, akan melakukan investasi, dan mayoritas dilakukan pada 30 hari pertama setelah register, dan sebagian kecil di bulan kedua. Di bulan selanjutnya peluangnya sangat kecil untuk bisa convert. Sehingga perlu dipastikan bagaimana journey investor tersebut lancar di bulan pertama, sehingga mau convert invest di DQLab Finance.
Selenjutnya perlu dilihat juga setelah invest pertama itu invest lagi di bulan bulan selanjutnya. Secara umum 30% investor akan invest lagi pada bulan berikutnya.
Pada bulan Febuari, conversion rate nya bagus, paling tinggi yakni 7.57%, secara jumlah juga paling banyak, tapi ketika dilihat retentionnya, hanya 16% yang invest pada bulan selanjutnya, hanya setengahnya dari kategori bulan bulan lainnya.
Perlu dianalisis lebih lanjut darimana dan profil dari investor di bulan Febuari sampai April 2020.
